In [2]:
from sklearn import datasets
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_mutual_info_score as AMI, adjusted_rand_score as ARI
from sklearn.decomposition import PCA
import warnings
import plotly.express as px
from ipywidgets import interact
warnings.filterwarnings("ignore")

In [3]:
iris = datasets.load_digits()
x = scale(iris.data, axis=0)
y = iris.target

In [4]:
x.shape

(1797, 64)

In [5]:
ln = len(set(y))
ln

10

In [6]:
class MyKMeans(KMeans):
    
    def __init__(self, init, n_clusters):
        super().__init__(init=init, n_clusters=n_clusters, n_init=10)
    
    def check(self, x, y):
        result = self.fit(x).labels_
        print(f'ARI: {ARI(y, result)}, AMI: {AMI(y, result)}')
    
    def check_plot(self, x, y):
        %time self.check(x,y)
        result = self.fit(x)
        x_new = list(zip(*tuple(x)))
        lab = tuple(map(lambda x: f'Class {x}', result.labels_))
        
        @interact(fir_coord=(0, 63), second_coord=(0,63))
        def plot(fir_coord=0, second_coord=1):
            if fir_coord == second_coord:
                print('Координаты должны быть различны')
                return
            
            # тут надо исправить
            t = px.scatter(x=x_new[fir_coord],y=x_new[second_coord],color=lab)
            t.show()

In [8]:
km = MyKMeans(init='k-means++', n_clusters=ln)
km.check_plot(x, y)

ARI: 0.5625724008412156, AMI: 0.6941055490338277
CPU times: total: 1.42 s
Wall time: 195 ms


interactive(children=(IntSlider(value=0, description='fir_coord', max=63), IntSlider(value=1, description='sec…

In [8]:
km = MyKMeans(init='random', n_clusters=ln)
km.check_plot(x, y)

ARI: 0.4645301922808515, AMI: 0.6197752489830856
CPU times: total: 1.66 s
Wall time: 512 ms


interactive(children=(IntSlider(value=0, description='fir_coord', max=63), IntSlider(value=1, description='sec…

In [11]:
pca = PCA(n_components=ln).fit(x)

In [12]:
km = MyKMeans(init=pca.components_, n_clusters=ln)
km.check_plot(x, y)

ARI: 0.5570729572305355, AMI: 0.6774313910031146
CPU times: total: 312 ms
Wall time: 43.9 ms


interactive(children=(IntSlider(value=0, description='fir_coord', max=63), IntSlider(value=1, description='sec…

In [ ]:
$dest_dir="C:\hadoop"
$url = "https://dlcdn.apache.org/hadoop/common/hadoop-3.3.1/hadoop-3.3.1.tar.gz"
$client = new-object System.Net.WebClient
$client.DownloadFile($url,$dest_dir+"\hadoop-3.3.1.tar.gz")